In [96]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

In [10]:
df = pd.read_csv("logs_limpios.csv")

In [11]:
text_list = df["Preguntas"].to_list()

> ### 1: ***Stemizar y sacar las stopwords del texto***

In [147]:
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import nltk
from nltk.corpus import stopwords

stemmer = SnowballStemmer('spanish')
cachedStopWords = stopwords.words('spanish')


def clean_text(text_list):
    for i in range(len(text_list)):
        text = ' '.join([stemmer.stem(word) for word in text_list[i].split() if word not in cachedStopWords])
        text_list[i] = text
    return text_list

In [149]:
text_list = clean_text(text_list)

In [150]:
vectorizer = TfidfVectorizer()

In [151]:
data = vectorizer.fit_transform(text_list)

In [152]:
svd = TruncatedSVD(n_components=20)
svd_truncated = svd.fit_transform(data)

In [ ]:
# method = 'exact' corre un algorito más exacto pero de complejidad O(N^2)

data_2d = TSNE(n_components=2, method='exact', early_exaggeration=1, perplexity=50, n_iter=3000)
data_2d = data_2d.fit_transform(svd_truncated)

In [154]:
from sklearn.cluster import DBSCAN

In [161]:
clustering = DBSCAN(eps=3, min_samples=15).fit_predict(data_2d)

------

> ### 5: ***Preparación de la data***

---

In [162]:
pos_x = [x[0] for x in data_2d]
pos_y = [x[1] for x in data_2d]

In [163]:
dic_plot = {}

for i in range(len(clustering)):
    if clustering[i] not in dic_plot:
        dic_plot[clustering[i]] = [[pos_x[i]],[pos_y[i]], [clustering[i]]]
    # Agrego la posición en X
    dic_plot[clustering[i]][0].append(pos_x[i])  
    # Agrego la posición en Y
    dic_plot[clustering[i]][1].append(pos_y[i])    
    # Agrego el número de "cluster"
    dic_plot[clustering[i]][2].append(clustering[i])

In [164]:
dic_plot.keys()

dict_keys([0, 1, 2, 18, 3, -1, 4, 5, 6, 7, 10, 16, 15, 11, 8, 9, 12, 13, 14, 17])

In [165]:
import plotly.offline
import plotly.graph_objs as go

plot = []
for intenciones in dic_plot:
    trace = go.Scatter(
        x = dic_plot[intenciones][0],
        y = dic_plot[intenciones][1],
        text = dic_plot[intenciones][2],
        mode = 'markers',
        marker = dict(
            size = 7
        )
    )
    plot.append(trace)

In [166]:
plotly.offline.plot(plot)

'file:///Users/nicolas.bugliot/Documents/Trabajo/ML/temp-plot.html'